In [6]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import shutil
from tqdm import tqdm

In [8]:
path_root = '../character_GRAY_clean/'

os.makedirs('src/char_classification/digits', exist_ok=True)
os.makedirs('src/char_classification/alphas', exist_ok=True)

digits = ['0','1','2','3','4','5','6','7','8','9','Background']

for path in tqdm(os.listdir(path_root)):
    if path == '.DS_Store':
        continue
    if path in digits:
        shutil.copytree(f'{path_root}/{path}', f'src/char_classification/digits/{path}')
    else:
        shutil.copytree(f'{path_root}/{path}', f'src/char_classification/alphas/{path}')

100%|██████████| 33/33 [00:06<00:00,  4.96it/s]


# Test model

In [43]:
from src.char_classification.model import CNN_Model
import cv2
import numpy as np
CHAR_CLASSIFICATION_WEIGHTS = './src/weights/weight_character.h5'
recogChar = CNN_Model(trainable=False).model
recogChar.load_weights(CHAR_CLASSIFICATION_WEIGHTS)

In [54]:
ALPHA_DICT = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'K', 9: 'L', 10: 'M', 11: 'N', 12: 'P',
              13: 'R', 14: 'S', 15: 'T', 16: 'U', 17: 'V', 18: 'X', 19: 'Y', 20: 'Z', 21: '0', 22: '1', 23: '2', 24: '3',
              25: '4', 26: '5', 27: '6', 28: '7', 29: '8', 30: '9', 31: "Background"}

characters = cv2.imread('char1.png', 0)
# characters = cv2.resize(characters, (28, 28))
characters = characters[np.newaxis, ...]
print(characters.shape)


(1, 28, 28)


In [55]:
result = recogChar.predict_on_batch(characters)
result_idx = np.argmax(result, axis=1)

candidates = []
for i in range(len(result_idx)):
    if result_idx[i] == 31:  # if is background or noise, ignore it
        continue
    candidates.append((ALPHA_DICT[result_idx[i]], coordinates[i]))

2022-08-11 09:47:17.138953: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-11 09:47:17.146857: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [58]:
result_idx

array([31])

# Test model detect

In [59]:
import torch
import numpy as np
import cv2

model_detect = torch.hub.load('ultralytics/yolov5', 'custom', path='src/weights/plate_yolo10k.pt', force_reload=True)  # local model

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /Users/NghiaKhang/.cache/torch/hub/master.zip
YOLOv5 🚀 2022-8-11 Python-3.9.9 torch-1.10.2 CPU

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


In [61]:
img = cv2.imread('car.jpg')
results = model_detect(img)
t = results.pandas().xyxy[0]
bbox = np.int32(np.array(t)[:,:4][np.argmax(np.array(t)[:,4])])

In [63]:
results.save()

Saved 1 image to runs/detect/exp


# Threshold

In [70]:
import matplotlib.pyplot as plt

In [98]:
img = cv2.imread('7.png', 0)
_, mask = cv2.threshold(img, 15, 255, cv2.THRESH_BINARY)
contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

l = []
for c in contours:
    area = cv2.contourArea(c)
    x, y, w, h = cv2.boundingRect(c)
    l.append((x,y,w,h,area))

l.pop(np.argmax(np.array(l)[:, -1]))
for bb in l:
    x, y, w, h = bb[:4]
    points = np.array([(x, y), (x + w, y), (x + w, y + h), (x, y + h)])
    cv2.fillPoly(img, pts = [points], color = (0, 0, 0))
cv2.imwrite('7_thresold.png', img)
print(l)

[(8, 22, 1, 2, 0.0), (7, 14, 1, 2, 0.0)]


2

In [94]:
l

[(8, 22, 1, 2, 0.0), (7, 14, 1, 2, 0.0)]

# Find y max

In [99]:
coord = [('2', (81, 26)), ('6', (12, 28)), ('1', (11, 58)), ('2', (78, 67)), ('2', (78, 105)), ('T', (8, 112)), ('3', (7, 143)), ('2', (75, 146))]

In [100]:
coord_arr = np.array(coord)

/var/folders/wy/gq4jkwhd5lvgbwr025mptx2h0000gn/T/ipykernel_22780/2810763986.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  coord_arr = np.array(coord)


In [127]:
l = coord_arr[:,1]

In [131]:
lst = []
for l_sub in l:
    lst.append(list(l_sub))

list

In [132]:
lst

[[81, 26],
 [12, 28],
 [11, 58],
 [78, 67],
 [78, 105],
 [8, 112],
 [7, 143],
 [75, 146]]

In [134]:
max(np.array(lst)[:, 0])

81